In [1]:

#ABM5

#This file loads the agents, locations and network connections generated by the file NetworkModel, to
#gether with the intial distributions and transition matrices generated by the file MarkovModel, and
#simulates an epidemic. Note that the population size N is determined within the file NetworkModel. I
#n this file, one can set the length of the simulation in weeks and the number of initial seeds. Note
#that the simulation starts on a Sunday and follows the SEIRD framework.

import numpy as np
import random
import math
import matplotlib.pyplot as plt
import xlsxwriter
from openpyxl import load_workbook

def multinoulli(problist):
    p = random.randint(1,np.sum(problist))
    summ = 0
    for i in range(problist.size):
        summ = summ + problist[i]
        if summ >= p:
            return i

print('Loading agents and locations...')

class Agent:
  def __init__(self, agetyp, age, location):
    self.agetyp = agetyp
    self.age = age
    self.location = location

agentworkbook = load_workbook(filename='Agents/Agents.xlsx')
agentsheet = agentworkbook.active

N = agentsheet.max_row

P = [[0, 0] for i in range(N)]

for i in range(N):
    P[i] = Agent(agentsheet.cell(row=i+1, column=2).value,agentsheet.cell(row=i+1, column=3).value,0)

LocationListAgent = [ [ [] for j in range(14) ] for i in range(N) ]
    
for i in range(N):
    for j in range(14):        
        LocationListAgent[i][j] = list(map(int,agentsheet.cell(row=i+1, column=j+4).value.split(',')))
        
class Location:
  def __init__(self, typ, coord, who):
    self.typ = typ
    self.coord = coord
    self.who = who
    
locationworkbook = load_workbook(filename='Locations/Locations.xlsx')
locationsheet = locationworkbook.active

M = locationsheet.max_row

L = [ [ 0, [0,0], [] ] for i in range(M) ]

for j in range(M):
    L[j] = Location(locationsheet.cell(row=j+1, column=2).value,list(map(int,locationsheet.cell(row=j+1, column=3).value.split(','))),[])

print('Loading initial distribution and transition matrices...')

Init = [ [ 0 for i in range(14) ] for j in range(3) ]

Init[0] = np.genfromtxt('Initial_Distributions/Init_child.csv', delimiter=',', dtype = 'int')
Init[1] = np.genfromtxt('Initial_Distributions/Init_adult.csv', delimiter=',', dtype = 'int')
Init[2] = np.genfromtxt('Initial_Distributions/Init_retired.csv', delimiter=',', dtype = 'int')

Trans = [ [ [ [ 0 for i in range(14) ] for j in range(14) ] for k in range(3) ] for t in range(7*144) ]

for t in range(7*144):
    Trans[t][0] = np.genfromtxt('Transition_Matrices/Trans_child_' + str(t) + '.csv', delimiter=',', dtype = 'int')
    Trans[t][1] = np.genfromtxt('Transition_Matrices/Trans_adult_' + str(t) + '.csv', delimiter=',', dtype = 'int')
    Trans[t][2] = np.genfromtxt('Transition_Matrices/Trans_retired_' + str(t) + '.csv', delimiter=',', dtype = 'int')

print('Loading pathogenic data...')

virusworkbook = load_workbook(filename='Data/Sars-CoV-2.xlsx')
virussheet = virusworkbook.active

incubationperiod = virussheet.cell(row=1, column=1).value #Incubation period, in units of 10 minutes
infectiousperiod = virussheet.cell(row=2, column=1).value #Infectious period, in units of 10 minutes

pdeath = [0 for i in range(10)]

for i in range(10):
    pdeath[i] = virussheet.cell(row=5 + i, column=1).value #Probabilities of death across 10-year age intervals

#If two individuals, one susceptible and one infectious, spend ten minutes together in location j, th
#en prob[j] is the average number of times, out of 10000 trials, that the susceptible will become inf
#ected:

prob = [0 for j in range(13)]

for j in range(13):
    prob[j] = virussheet.cell(row=17 + j, column=1).value #Transmission probabilities

print('Simulating epidemic...')

T = 10 #Length of simulation in weeks, starting at midnight on a Saturday

numberofinitialseeds = 10

#The active status of each individual will be recorded by traj. Here traj[0][i] will refer to the cur
#rent ten minute interval, traj[1][i] to the next:

traj = [ [ 0 for i in range(N) ] for t in range(2) ]

#The health status of each individual will be recorded by H. This follows the SEIRD framework with th
#e possibilities codified in the file FormatHealth. #Here H[0][i] will refer to the current ten minut
#e interval, H[1][i] to the next:

H = [ [ 0 for i in range(N) ] for t in range(2) ]

for i in range(numberofinitialseeds):
    H[0][random.randrange(N)] = 2

tenminsincubating = [ 0 for i in range(N) ] #Records how long individual i has been incubating
tenminsinfectious = [ 0 for i in range(N) ] #Records how long individual i has been infectious

deaths = 0 #The total number of deaths

infectious = [ 0 for i in range(T*7*144) ] #How many individuals are infectious at each time

#Using the initial distribution individuals are now assigned starting locations:

for i in range(N):
    traj[0][i] = multinoulli(Init[P[i].agetyp])
    randloc = random.randrange(len(LocationListAgent[i][traj[0][i]]))
    P[i].location = LocationListAgent[i][traj[0][i]][randloc]
    L[P[i].location].who.append(i)

#Finally the epidemic can be simulated. In each ten minute interval, the code first loops through all
#locations, in which the health status of individuals is updated, after which it loops through all in
#dividuals, in which the locations of individuals are updated. Note that individuals only change loca
#tion if the Markov chain generates a new activity. For example, once an individual is inside a shop,
#they cannot move directly to another shop.
    
t = 0
for week in range(T):    
    for tenminute in range(7*144):
        t = week*7*144 + tenminute
        for j in range(M):
            for i in L[j].who:
                if (H[0][i] == 1):
                    tenminsincubating[i] = tenminsincubating[i] + 1
                    if (tenminsincubating[i] < incubationperiod):
                        H[1][i] = 1
                    if (tenminsincubating[i] == incubationperiod):
                        H[1][i] = 2
                if (H[0][i] == 2):
                    infectious[t] = infectious[t] + 1
                    tenminsinfectious[i] = tenminsinfectious[i] + 1
                    if (tenminsinfectious[i] < infectiousperiod):
                        H[1][i] = 2
                        for k in L[j].who:
                            if(H[0][k] == 0):
                                if (random.randrange(10000)+1 <= prob[L[j].typ]):
                                    H[1][k] = 1
                    if (tenminsinfectious[i] == infectiousperiod):
                        if (random.randrange(100)+1 <= pdeath[P[i].age//10]):
                            H[1][i] = 4
                            deaths = deaths + 1
                        else:
                            H[1][i] = 3
                if (H[0][i] == 3):
                    H[1][i] = 3
                if (H[0][i] == 4):
                    H[1][i] = 4
        for i in range(N):
            H[0][i] = H[1][i]
            traj[1][i] = multinoulli(Trans[tenminute][P[i].agetyp][traj[0][i],:])
            if ( traj[1][i] != traj[0][i] ):
                L[P[i].location].who.remove(i)
                randloc = random.randrange(len(LocationListAgent[i][traj[1][i]]))
                P[i].location = LocationListAgent[i][traj[1][i]][randloc]
                L[P[i].location].who.append(i)
            traj[0][i] = traj[1][i]
    print('Week', week+1, '/', T, 'complete')

print('Deaths:', deaths)
    
print('Saving results...')
    
workbook = xlsxwriter.Workbook('Results/ABM_Results.xlsx') 
worksheet = workbook.add_worksheet()

for t in range(T*7*144):
    worksheet.write(t,0, infectious[t])
    
workbook.close()

print('Done.')

Loading agents and locations...
Loading initial distribution and transition matrices...
Loading pathogenic data...
Simulating epidemic...
Week 1 / 10 complete
Week 2 / 10 complete
Week 3 / 10 complete
Week 4 / 10 complete
Week 5 / 10 complete
Week 6 / 10 complete
Week 7 / 10 complete
Week 8 / 10 complete
Week 9 / 10 complete
Week 10 / 10 complete
Deaths: 7
Saving results...
Done.
